3/5/2022: We ran 1x1 cadences and 174x8 would take forever. Let's hack the fits files to make them appear to be a 174x8. 

To do:
-  change NEXP in header (implemented, imp)
-  change FCADENCE in header (imp)
-  change DESMODE in header (imp)
- CHANGE TO REPEAT DESMODE AND SKIES AND STANDARDS LIKE EXAMPLE (1392 TIMES) (imp)
- **change cadence data column to dark_174x8**
-  **check structure of a 174x8**
- **change hdu 2 data to match structure (I think hdu1 is fine now?)**

In [18]:
#I need the header to look at NEXP
from astropy.io import fits
import glob
import numpy as np
from astropy.table import Table

#Collect files to be hacked
survey_files = glob.glob('./LCO_commissioning_designs/SCVZ_design*.fits')
print(survey_files)
print("")

cadence_name = 'dark_100x8'

if cadence_name == 'dark_174x8':
    example = "RM_survey2.fits" #a 174x8
else:
    example = "rsFieldAssignmentsFinal-zeta-4-lco-112357.fits" #a 100x8!


######CHECK THESE BEFORE YOU GO!!!#######
epoch = int(cadence_name.split("_")[1].split("x")[0])
exposures = int(cadence_name.split("_")[1].split("x")[0])*int(cadence_name.split("_")[1].split("x")[1])
print("Cadence name: ", cadence_name)
print("epoch: ", epoch)
print("exposures: ", exposures)
print("Using example: ", example)

['./LCO_commissioning_designs/SCVZ_design1.fits', './LCO_commissioning_designs/SCVZ_design3.fits', './LCO_commissioning_designs/SCVZ_design2.fits']

Cadence name:  dark_100x8
epoch:  100
exposures:  800
Using example:  rsFieldAssignmentsFinal-zeta-4-lco-112357.fits


In [19]:
for i in survey_files:
    print(i)
    head = fits.open(i)[0].header
    print(head['RACEN'])
    print(head['DECCEN'])

./LCO_commissioning_designs/SCVZ_design1.fits
90.0
-66.5605
./LCO_commissioning_designs/SCVZ_design3.fits
90.0
-66.5605
./LCO_commissioning_designs/SCVZ_design2.fits
90.0
-66.5605


## Edit Header, Save

HACKED FOR LCO!!!!!!!!!! CHANGE NAMING SCHEME AND CHECK NEXP, EXPOSURES, and CADENCE NAME

In [20]:
# #Open the example #NECESSARY, DO NOT DELETE
examp = fits.open(example)

In [21]:
#columns to change
#key= col name
#item = shape (#,)

#THIS ASSUMES THAT ALL OF THEM HAD 50 SKIES and 70 STANDARDS!!!!!!
#CHECK THAT OR ELSE YOU COULD DESTROY SDSS MEG

change_cols = {'allowed': epoch, 'mags_allowed': epoch,'bright_allowed': epoch, 
           "robotID":exposures, "holeID":exposures,'equivRobotID':exposures, 
           'target_skybrightness':exposures, 'field_skybrightness':exposures, 
           'scienceRobotID':exposures}

for i in survey_files:
    print(i)
    surv_file = fits.open(i)
    print(surv_file[0].header['RACEN'])
    print(surv_file[0].header['DECCEN'])
    surv_file[0].header['DESMODE'] = examp[0].header['DESMODE']
    surv_file[0].header['ACNUM0'] = examp[0].header['ACNUM0']
    surv_file[0].header['ACNUM1'] = examp[0].header['ACNUM1']
    surv_file[0].header['ACNUM2'] = examp[0].header['ACNUM2']
    surv_file[0].header['ACNUM3'] = examp[0].header['ACNUM3']
    surv_file[0].header['FCADENCE'] = cadence_name
    surv_file[0].header['NEXP'] = exposures
    
    
    data1 = Table.read(i, hdu=1, format='fits')
    data2 = Table.read(i, hdu=2, format='fits')

    data1['tag'] = np.full(len(data1['catalogid']), '0.3.5') #same regardless of field
    data1['plan'] = np.full(len(data1['catalogid']), '0.5.4') #same
    data1['cadence'] = np.full(len(data1['catalogid']), cadence_name) #same

    #mapper stuff only correlates to the science targets
    data1['mapper'] = np.full(len(data1['catalogid']), "   ") #same
    data1['mapper'][data1['category'] == 'science'] = 'BHM'

    data2['nexps'][data2['nexps'] == 1] = exposures
    data2['nepochs'][data2['nepochs'] == 1] = epoch
    
    for key in change_cols.keys():
        new_column = []
        for j in range(len(data2[key])):
            new_entry = np.full(change_cols[key], data2[key][j])
            new_column.append(new_entry)
        data2[key] = new_column
    
    hdu_list = fits.HDUList([fits.PrimaryHDU(header =surv_file[0].header),
    fits.table_to_hdu(data1),
    fits.table_to_hdu(data2)])
    
    hdu_list.writeto('./LCO_commissioning_designs/{}_final_{}.fits'.format(i.split("_")[2].split("/")[1], i.split("_")[-1].split(".")[0]), overwrite =True)
    
    print("SAVING HACKED FILE.")


./LCO_commissioning_designs/SCVZ_design1.fits
90.0
-66.5605
SAVING HACKED FILE.
./LCO_commissioning_designs/SCVZ_design3.fits
90.0
-66.5605
SAVING HACKED FILE.
./LCO_commissioning_designs/SCVZ_design2.fits
90.0
-66.5605
SAVING HACKED FILE.


In [22]:
survey_filesHACKED = glob.glob('./LCO_commissioning_designs/SCVZ_HACKED*.fits')
print(survey_filesHACKED)
i = 0
print(survey_filesHACKED[i])
head = fits.open(survey_filesHACKED[i])[0].header
data1 = Table.read(survey_filesHACKED[i], hdu=1, format='fits')
data2 = Table.read(survey_filesHACKED[i], hdu=2, format='fits')
print(head['RACEN'])
print(head['DECCEN'])
len(head['DESMODE'])

[]


IndexError: list index out of range

In [4]:
head

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                    8 / array data type                                
NAXIS   =                    0 / number of array dimensions                     
EXTEND  =                    T                                                  
COMMENT   FITS (Flexible Image Transport System) format is defined in 'Astronomy
COMMENT   and Astrophysics', volume 376, page 359; bibcode: 2001A&A...376..359H 
STRATVER= '1.4.4   '           / robostrategy version                           
SCHEDVER= '0.11.1  '           / roboscheduler version                          
MUGATVER= '1.3.2   '           / mugatu version                                 
KAIJUVER= '1.2.2   '           / kaiju version                                  
FIELDID =                    1 / field identification number                    
RACEN   =                  90. / RA J2000 center of field (deg)                 
DECCEN  =             -66.56